In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input,Conv2D,MaxPool2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
#from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D

import tensorflow.keras as keras

from tensorflow.keras import layers


from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.optimizers import Adam,SGD

In [18]:
(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)
X_train,X_test=X_train/255,X_test/255

aug_data=ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3)
aug_data.fit(X_train)

In [19]:

inputs=keras.Input(shape=(32,32,3))

a=Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(inputs)

a=Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)
a=(MaxPool2D(pool_size=(3,3),strides=(2,2)))(a)


a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(a)

a=Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

#inception-1
b=Conv2D(64, (1,1), padding='same',strides=1,activation=LeakyReLU(alpha=0.1))(a)
b=Conv2D(64, (3,3), padding='same',strides=1 ,activation=LeakyReLU(alpha=0.1))(b)
b=Conv2D(64, (3,3), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(b)


c=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
c=Conv2D(64, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(c)

d=MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
d=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(d)

e=Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=concatenate([b, c, d,e],axis=3)

#inception-2
f=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
f=Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)

g=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
g=Conv2D(128, (1,7), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(g)
g=Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(g)

h=MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
h=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

i=Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

a=concatenate([f,g,h,i],axis=3)

#inception-3
j=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

j=Conv2D(256, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
k=Conv2D(256, (1,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
l=Conv2D(256, (3,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)

m=Conv2D(256, (1,1), padding='same', activation=LeakyReLU(alpha=0.1))(a)
n=Conv2D(256, (1,3), padding='same', activation=LeakyReLU(alpha=0.1))(m)
o=Conv2D(256, (3,1), padding='same', activation=LeakyReLU(alpha=0.1))(m)

p=MaxPool2D((3,3), strides=(1,1), padding='same')(a)
p=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(p)

q=Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=concatenate([k,l,n,o,p,q],axis=3)

a=MaxPooling2D((8,8), strides=(1,1), padding='same')(a)


a=Flatten()(a)

a=Dense(units=256,activation=LeakyReLU(alpha=0.3))(a)

outputs=Dense(y_train.shape[1],activation="softmax")(a)

model=keras.Model(inputs,outputs)

adam=Adam(learning_rate=0.001,clipnorm=0.7)

model.compile(loss="categorical_crossentropy",optimizer=adam,metrics=['accuracy'])


call=EarlyStopping(monitor="val_loss",verbose=1,mode="auto",patience=10,restore_best_weights="true")

checkpoint=ModelCheckpoint('Inception_adam_non_regularization_final.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True,save_weights_only=True,model='auto',period=1)


In [20]:
model.fit_generator(aug_data.flow(X_train,y_train,batch_size=128),steps_per_epoch=len(X_train)/128,epochs=80,validation_data=(X_test,y_test),verbose=1,callbacks=[call,checkpoint])

Epoch 1/80
391/390 [==============================] - ETA: 0s - loss: 4.1575 - accuracy: 0.0534
Epoch 00001: val_accuracy improved from -inf to 0.10890, saving model to Inception_adam_non_regularization_final.hdf5
391/390 [==============================] - 28s 73ms/step - loss: 4.1575 - accuracy: 0.0534 - val_loss: 3.7577 - val_accuracy: 0.1089
Epoch 2/80
391/390 [==============================] - ETA: 0s - loss: 3.6746 - accuracy: 0.1252
Epoch 00002: val_accuracy improved from 0.10890 to 0.16050, saving model to Inception_adam_non_regularization_final.hdf5
391/390 [==============================] - 28s 71ms/step - loss: 3.6746 - accuracy: 0.1252 - val_loss: 3.4751 - val_accuracy: 0.1605
Epoch 3/80
391/390 [==============================] - ETA: 0s - loss: 3.4174 - accuracy: 0.1730
Epoch 00003: val_accuracy improved from 0.16050 to 0.19440, saving model to Inception_adam_non_regularization_final.hdf5
391/390 [==============================] - 28s 71ms/step - loss: 3.4174 - accuracy: 0.

In [21]:
y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

In [22]:
from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))

              precision    recall  f1-score   support

           0       0.74      0.67      0.71       100
           1       0.53      0.56      0.55       100
           2       0.38      0.30      0.33       100
           3       0.34      0.30      0.32       100
           4       0.36      0.24      0.29       100
           5       0.46      0.53      0.50       100
           6       0.47      0.55      0.50       100
           7       0.52      0.57      0.54       100
           8       0.42      0.55      0.48       100
           9       0.85      0.47      0.61       100
          10       0.27      0.16      0.20       100
          11       0.37      0.24      0.29       100
          12       0.47      0.38      0.42       100
          13       0.28      0.56      0.37       100
          14       0.36      0.32      0.34       100
          15       0.41      0.28      0.33       100
          16       0.54      0.38      0.44       100
          17       0.51    

In [23]:
model.summary()

Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_96 (Conv2D)              (None, 16, 16, 32)   896         input_4[0][0]                    
__________________________________________________________________________________________________
conv2d_97 (Conv2D)              (None, 16, 16, 32)   9248        conv2d_96[0][0]                  
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 16, 16, 32)   9248        conv2d_97[0][0]                  
_______________________________________________________________________________________

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense,Flatten,Input,Conv2D,MaxPool2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.merge import concatenate
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
#from tensorflow.keras.layers import concatenate
from keras.layers import Conv2D

import tensorflow.keras as keras

from tensorflow.keras import layers



from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.layers import LeakyReLU

from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

from keras.optimizers import Adam,SGD


(X_train,Y_train),(X_test,Y_test)=cifar100.load_data()
y_train,y_test=to_categorical(Y_train),to_categorical(Y_test)
X_train,X_test=X_train/255,X_test/255


inputs=keras.Input(shape=(32,32,3))

a=layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(inputs)

a=layers.Conv2D(filters=32,kernel_size=(3,3),padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)
a=layers.MaxPool2D(pool_size=(3,3),strides=(2,2))(a)


a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=2,activation=LeakyReLU(alpha=0.1))(a)

a=layers.Conv2D(filters=32, kernel_size=(3,3), padding="same",strides=1,activation=LeakyReLU(alpha=0.1))(a)

#inception-1
b=layers.Conv2D(64, (1,1), padding='same',strides=1,activation=LeakyReLU(alpha=0.1))(a)
b=layers.Conv2D(64, (3,3), padding='same',strides=1 ,activation=LeakyReLU(alpha=0.1))(b)
b=layers.Conv2D(64, (3,3), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(b)


c=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
c=layers.Conv2D(64, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(c)

d=layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
d=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(d)

e=layers.Conv2D(64, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=layers.concatenate([b, c, d,e],axis=3)

#inception-2
f=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
f=layers.Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (1,7), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)
f=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(f)

g=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)
g=layers.Conv2D(128, (1,7), padding='same', strides=1,activation=LeakyReLU(alpha=0.1))(g)
g=layers.Conv2D(128, (7,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(g)

h=layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(a)
h=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

i=layers.Conv2D(128, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(h)

a=layers.concatenate([f,g,h,i],axis=3)

#inception-3
j=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

j=layers.Conv2D(256, (3,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
k=layers.Conv2D(256, (1,3), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)
l=layers.Conv2D(256, (3,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(j)

m=layers.Conv2D(256, (1,1), padding='same', activation=LeakyReLU(alpha=0.1))(a)
n=layers.Conv2D(256, (1,3), padding='same', activation=LeakyReLU(alpha=0.1))(m)
o=layers.Conv2D(256, (3,1), padding='same', activation=LeakyReLU(alpha=0.1))(m)

p=layers.MaxPool2D((3,3), strides=(1,1), padding='same')(a)
p=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(p)

q=layers.Conv2D(256, (1,1), padding='same', strides=1, activation=LeakyReLU(alpha=0.1))(a)

a=layers.concatenate([k,l,n,o,p,q],axis=3)

a=layers.MaxPooling2D((8,8), strides=(1,1), padding='same')(a)


a=layers.Flatten()(a)

a=layers.Dense(units=256,activation=LeakyReLU(alpha=0.3))(a)

outputs=layers.Dense(y_train.shape[1],activation="softmax")(a)

model=keras.Model(inputs,outputs)

model.load_weights("../weights/Inception_adam_non_regularization_final.hdf5")

y_true = y_test.argmax(-1)
y_pred = model.predict(X_test).argmax(-1)

from sklearn.metrics import classification_report,accuracy_score,recall_score,precision_score
print(classification_report(Y_test,y_pred))
print("Accuracy is {}".format(accuracy_score(Y_test,y_pred)))
print("Precision is {}".format(precision_score(Y_test,y_pred,average="weighted")))
print("Recall is {}".format(recall_score(Y_test,y_pred,average="weighted")))


Using TensorFlow backend.


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
              precision    recall  f1-score   support

           0       0.70      0.72      0.71       100
           1       0.54      0.56      0.55       100
           2       0.37      0.30      0.33       100
           3       0.25      0.18      0.21       100
           4       0.25      0.16      0.20       100
           5       0.41      0.42      0.41       100
           6       0.51      0.47      0.49       100
           7       0.45      0.53      0.49       100
           8       0.41      0.53      0.46       100
           9       0.61      0.51      0.56       100
          10       0.26      0.14      0.18       100
          11       0.27      0.30      0.28       100
          12       0.44      0.46      0.45       100
          13       0.40      0.41      0.40       100
          14       0.40      0.35      0.37       100
          15     